# Twitter Named Entity Recognition Case Study

### About
Twitter is a microblogging and social networking service on which users post and interact with messages known as "tweets". Every second, on average, around 6,000 tweets are tweeted on Twitter, corresponding to over 350,000 tweets sent per minute, 500 million tweets per day.

### Problem statement 
Twitter wants to automatically tag and analyze tweets for better understanding of the trends and topics without being dependent on the hashtags that the users use. Many users do not use hashtags or sometimes use wrong or mis-spelled tags, so they want to completely remove this problem and create a system of recognizing important content of the tweets.

### Objective
Named Entity Recognition (NER) is an important subtask of information extraction that seeks to locate and recognise named entities.
We need to train models that will be able to identify the various named entities.

### Data
Dataset is annotated with 10 fine-grained NER categories: person, geo-location, company, facility, product,music artist, movie, sports team, tv show and other. Dataset was extracted from tweets and is structured in CoNLL format., in English language. Containing in Text file format.
The CoNLL format is a text file with one word per line with sentences separated by an empty line. The first word in a line should be the word and the last word should be the label.

In [2]:
# imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf

import warnings
warnings.filterwarnings('ignore')

In [23]:
import os
root_path = os.path.abspath(os.path.join(os.getcwd(),os.pardir))
data_path = os.path.join(root_path,'data')
train_data_path = os.path.join(data_path,'wnut 16.txt.conll')
test_data_path = os.path.join(data_path,'wnut 16test.txt.conll')

## Getting the data

In [3]:
# installing conllu library
!pip install conllu

In [25]:
# reading the training file
with open(train_data_path,'r') as f:
    train_raw = f.read()

In [39]:
from conllu import parse
# def extract_ner_from_conll(conll_data):
#     parsed_data = parse(conll_data)
#     ner_data = []

#     for sentence in parsed_data:
#         for token in sentence:
#             word = token.get('form')
#             label = token.get('ner')

#             if word and label:
#                 ner_data.append((word, label))

#     return ner_data

def extract_ner_from_conll(conll_data):
    # Split the data into sentences based on empty lines
    sentences = [sentence.strip() for sentence in conll_data.strip().split('\n\n')]
    ner_data = []

    for sentence in sentences:
        lines = sentence.split('\n')
        for line in lines:
            parts = line.split('\t')
            if len(parts) == 2:
                word = parts[0]
                label = parts[1]
                ner_data.append((word, label))

    return ner_data

In [41]:
len(extract_ner_from_conll(train_raw))

46469

In [36]:
train_raw.split("\n\n")[0].split("\n")[0].split("\t")

['@SammieLynnsMom', 'O']